In [2]:
from find_best_car import (
    _get_df_with_cost_to_own, 
    Column, 
    get_scored_df,
    get_cars,
    _fix_numeric_columns,
    _apply_de_discount,
    _filter_by_models,
    _filtered_cars
)
import pandas as pd
from numpy import nan


df = _filtered_cars()
df_scored = get_scored_df(
    only_mentioned_cars=False, de_discount=True, keep_columns=["id"]
)

df = pd.merge(df, df_scored.drop("name", axis=1), on="id", how="left")

models = "Peugeot 208,opel corsa,hyundai ioniq hybrid plugin,kia xceed,kia ceed,ford focus,vw golf,seat leon,skoda fabia,skoda octavia,toyota corolla,opel grandland x,subaru impreza,renault captur,renault clio,Peugeot 2008,ford puma,ford fiesta,Peugeot 3008,skoda karoq,skoda kodiaq,kia sportage,vw t roc,vw tiguan,subaru outback,hyundai tucson,seat ateca,vw t-cross,Mercedes A,Mercedes B,audi a1,audi a3,Bmw 1,honda civic,Toyota rav4,Peugeot 508,Opel insignia,ford kuga,tesla,VW ID.3".split(',')
df = _filter_by_models(df, models)

PRICE = 'Fahrzeugpreis für die Berechnung'
df = _apply_de_discount(df, PRICE)
df.head()

,name,url,id,image,processed date,CO2-Effizienzklasse,Aufladung,Baureihe_x,Karosserie,Anzahl der Schiebetüren serienmäßig/auf Wunsch,...,Insurance 2,Costs service,Costs running,Costs fixed,Costs depreciation,monthly costs,price,AGR,Total Score,Euro per score
0,Skoda Fabia 1.0 TSI Ambition DSG (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,315461,https://www.adac.de/_ext/itr/tests/ADAC40/Auto...,2020-10-11 23:55:14.787611,B,Turbo,Fabia (III) (ab 08/18),Schrägheck,- (Serie) / - (Aufpreis),...,11.86,0.15,2.90,0.81,6.50,17.16,29.55,NaN,118.264539,172.004222
1,Skoda Fabia 1.0 TSI Style DSG (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,315462,https://www.adac.de/_ext/itr/tests/ADAC40/Auto...,2020-10-11 23:55:14.837622,B,Turbo,Fabia (III) (ab 08/18),Schrägheck,- (Serie) / - (Aufpreis),...,11.86,0.15,2.90,0.81,6.42,17.16,28.96,NaN,118.723571,178.001721
2,Skoda Fabia Combi 1.0 TSI Ambition DSG (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,315467,https://www.adac.de/_ext/itr/tests/ADAC40/Auto...,2020-10-11 23:55:42.816931,B,Turbo,Fabia (III) Combi (ab 08/18),Kombi,- (Serie) / - (Aufpreis),...,12.30,0.15,2.83,0.62,6.36,17.42,28.79,NaN,117.833571,181.332024
5,Audi A1 Sportback 25 TFSI S tronic (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,314805,https://www.adac.de/_ext/itr/tests/ADAC40/Auto...,2020-10-11 23:57:33.980902,B,Turbo,A1 (GB) Sportback (ab 11/18),Schrägheck,- (Serie) / - (Aufpreis),...,10.78,0.38,3.18,1.30,6.12,16.00,26.88,0.0,207.304969,115.602632
6,Audi A1 Sportback 25 TFSI advanced S tronic (a...,https://www.adac.de/rund-ums-fahrzeug/autokata...,314807,https://www.adac.de/_ext/itr/tests/ADAC40/Auto...,2020-10-11 23:57:34.042918,B,Turbo,A1 (GB) Sportback (ab 11/18),Schrägheck,- (Serie) / - (Aufpreis),...,10.78,0.84,3.25,1.30,5.97,15.26,26.02,0.0,206.054969,121.982013


In [5]:
DEPRICATION = 'Wertverlust'
COSTS_OPERATION = 'Betriebskosten'
COSTS_FIX = 'Fixkosten'
COSTS_WORKSHOP = 'Werkstattkosten'
YEAR_TAX = 'KFZ-Steuer pro Jahr'
YEAR_INSURANCE = 'Haftpflichtbeitrag 100%'
YEAR_FULL_INSURANCE = 'Vollkaskobetrag 100% 500 Euro SB'

cols = [
    PRICE,
    DEPRICATION,
    COSTS_OPERATION,
    COSTS_FIX,
    COSTS_WORKSHOP,
    YEAR_TAX,
    YEAR_INSURANCE,
    YEAR_FULL_INSURANCE,
    Column.EURO_PER_SCORE,
    Column.TOTAL_SCORE
]

MONEY_5_YEARS = 'Money spent 5 years'
MONEY_5_YEARS_CAR_SOLD = 'Money spent 5 years, car sold'
MONEY_MONTHLY = 'Money spent monthly'
MONEY_MONTHLY_CAR_SOLD = 'Money spent monthly, car sold'

# double because 30k km per year
df[MONEY_5_YEARS] = df[PRICE] + 5 * ((df[COSTS_OPERATION] + df[COSTS_WORKSHOP]) * 2 + df[YEAR_TAX] + (df[YEAR_INSURANCE] + df[YEAR_FULL_INSURANCE])/2)

df[MONEY_5_YEARS_CAR_SOLD] = df[MONEY_5_YEARS] - (df[PRICE] - df[DEPRICATION] * 60)

df[MONEY_MONTHLY] = df[MONEY_5_YEARS] / 60
df[MONEY_MONTHLY_CAR_SOLD] = df[MONEY_5_YEARS_CAR_SOLD] / 60

ndf = df[['name', 'url', MONEY_5_YEARS_CAR_SOLD, MONEY_5_YEARS, MONEY_MONTHLY, MONEY_MONTHLY_CAR_SOLD] + cols]

# filter better than subaru

subaru = ndf[ndf.name.str.contains('Subaru Impreza 2.0i Exclusive')].iloc[0]
ndf = ndf[(ndf[Column.EURO_PER_SCORE] <= subaru[Column.EURO_PER_SCORE]) & (ndf[Column.TOTAL_SCORE] >= subaru[Column.EURO_PER_SCORE])]

ndf.to_excel('/tmp/out.xlsx')
ndf.head()

,name,url,"Money spent 5 years, car sold",Money spent 5 years,Money spent monthly,"Money spent monthly, car sold",Fahrzeugpreis für die Berechnung,Wertverlust,Betriebskosten,Fixkosten,Werkstattkosten,KFZ-Steuer pro Jahr,Haftpflichtbeitrag 100%,Vollkaskobetrag 100% 500 Euro SB,Euro per score,Total Score
5,Audi A1 Sportback 25 TFSI S tronic (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,22217.5,30162.5,502.708333,370.291667,23965.0,267.0,119.0,92.0,50.0,98.0,733.0,874.0,115.602632,207.304969
6,Audi A1 Sportback 25 TFSI advanced S tronic (a...,https://www.adac.de/rund-ums-fahrzeug/autokata...,23037.5,31432.5,523.875000,383.958333,25135.0,279.0,120.0,92.0,56.0,104.0,733.0,874.0,121.982013,206.054969
7,Audi A1 Sportback 25 TFSI S line S tronic (ab ...,https://www.adac.de/rund-ums-fahrzeug/autokata...,23727.5,32534.5,542.241667,395.458333,26207.0,290.0,122.0,92.0,56.0,106.0,733.0,874.0,127.829690,205.014969
8,Audi A1 Sportback 30 TFSI S tronic (ab 09/20),https://www.adac.de/rund-ums-fahrzeug/autokata...,22907.5,31351.5,522.525000,381.791667,25184.0,279.0,117.0,91.0,51.0,94.0,733.0,874.0,118.091534,213.258303
9,Audi A1 Sportback 30 TFSI advanced S tronic (a...,https://www.adac.de/rund-ums-fahrzeug/autokata...,23727.5,32620.5,543.675000,395.458333,26353.0,291.0,119.0,92.0,56.0,100.0,733.0,874.0,124.313463,211.988303
